In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"; # for GPU 1.
import pathlib
# import warnings
import random

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import add
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

pixel = 128
# batch_size = 128
batch_size = 128

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3) # channels = 3, colour RGB
    image = tf.image.resize(image, [pixel, pixel]) # resize to 128 x 128
    image /= 255.0  # normalize to [0,1] range, need 256.0 because float number, else 255 will be integer

    return image

In [3]:
def load_and_preprocess_image(path):
    print('load_and_preprocess_image:', path)
    image = tf.read_file(path)
    return preprocess_image(image)

In [4]:
def resLyr(inputs,
           numFilters=16,
           kernelSize=3,
           strides=1,
           activation='relu',
           batchNorm=True,
           convFirst=True,
           lyrName=None):
    convLyr = Conv2D(numFilters,
                     kernel_size=kernelSize,
                     strides=strides,
                     padding='same',
                     kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4),
                     name=lyrName + '_conv' if lyrName else None)

    x = inputs
    if convFirst:
        x = convLyr(x)
        if batchNorm:
            x = BatchNormalization(name=lyrName + '_bn' if lyrName else None)(x)

        if activation is not None:
            x = Activation(activation, name=lyrName + '_' + activation if lyrName else None)(x)
    else:
        if batchNorm:
            x = BatchNormalization(name=lyrName + '_bn' if lyrName else None)(x)

        if activation is not None:
            x = Activation(activation, name=lyrName + '_' + activation if lyrName else None)(x)
        x = convLyr(x)

    return x

In [5]:
def resBlkV1(inputs,
             numFilters=16,
             numBlocks=4,
             downSampleOnFirst=True,
             names=None):
    x = inputs
    for run in range(0, numBlocks):
        strides = 1
        blkStr = str(run + 1)
        if downSampleOnFirst and run == 0:
            strides = 2

        y = resLyr(inputs=x,
                   numFilters=numFilters,
                   strides=strides,
                   lyrName=names + '_Blk' + blkStr + '_Res1' if names else None)
        y = resLyr(inputs=y,
                   numFilters=numFilters,
                   activation=None,
                   lyrName=names + '_Blk' + blkStr + '_Res2' if names else None)

        if downSampleOnFirst and run == 0:
            x = resLyr(inputs=x,
                       numFilters=numFilters,
                       kernelSize=1,
                       strides=strides,
                       activation=None,
                       batchNorm=False,
                       lyrName=names + '_Blk' + blkStr + '_lin' if names else None)

        x = add([x, y],
                name=names + '_Blk' + blkStr + '_add' if names else None)

        x = Activation('relu', name=names + '_Blk' + blkStr + '_relu' if names else None)(x)

    return x

In [6]:
def createResNetV1(inputShape=(128, 128, 3),
                   numberClasses=3):
    inputs = Input(shape=inputShape)
    v = resLyr(inputs, numFilters=16, kernelSize=5, lyrName='Inpt')
    v = resBlkV1(inputs=v,
                 numFilters=16,
                 numBlocks=5,
                 downSampleOnFirst=False,
                 names='Stg1')
    v = resBlkV1(inputs=v,
                 numFilters=32,
                 numBlocks=5,
                 downSampleOnFirst=True,
                 names='Stg2')
    v = resBlkV1(inputs=v,
                 numFilters=64,
                 numBlocks=5,
                 downSampleOnFirst=True,
                 names='Stg3')

    v = AveragePooling2D(pool_size=8,
                         name='AvgPool')(v)
    v = Flatten()(v)
    outputs = Dense(numberClasses,
                    activation='softmax',
                    kernel_initializer='he_normal')(v)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.002),
                  metrics=['accuracy'])
    return model


In [7]:
def createModel(target_size=(128, 128)):
    model = createResNetV1(inputShape=(target_size[0], target_size[1], 3))

    return model

In [8]:

# Setup the models
def printSample(all_image_paths):
    img_path = all_image_paths[0]
    image_path = img_path
    img_raw = tf.io.read_file(img_path)
    # print(repr(img_raw)[:100]+' ...')
    img_tensor = tf.image.decode_png(img_raw, channels=3)
    img_tensor = tf.image.resize_image_with_crop_or_pad(img_tensor, 128, 128)
    print(img_tensor.shape, ' ', img_tensor.dtype)

    img = mpimg.imread(image_path)
    imgplot = plt.imshow(img)
    plt.show()

    img_final = tf.image.resize(img_tensor, [128, 128])
    img_final = tf.cast(img_final, tf.float32)
    img_final = img_final / 255.0
    print(img_final.shape, ' ', img_final.numpy().min(), ' ', img_final.numpy().max())

In [9]:
def lrSchedule(epoch):
    lr = 1e-3
    if epoch > 160:
        lr *= 0.5e-3
    elif epoch > 140:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1

    print('Learning rate:', lr)
    return lr

In [ ]:
# def main():
    target_size = (pixel, pixel)

    seed = 29

    tdf = pd.read_csv("test_set.csv")
    vdf = pd.read_csv("v_set.csv")
    model = createModel(target_size)

    print('model summary:', model.summary())

    modelname = 'full_resnet_v2'
    filepath = modelname + ".hdf5"
    checkpoint = ModelCheckpoint(filepath,
                                 monitor='val_acc',
                                 verbose=0,
                                 save_best_only=True,
                                 mode='max')

    # Log the epoch detail into csv
    csv_logger = CSVLogger(modelname + '.csv')
    # callbacks_list  = [checkpoint,csv_logger]

    LRScheduler = LearningRateScheduler(lrSchedule)
    callbacks_list = [checkpoint, csv_logger, LRScheduler]

    datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=20,
        zoom_range=0.10,
        shear_range=0.15,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest')

    vdatagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0,
        height_shift_range=0,
        rotation_range=0,
        zoom_range=0,
        shear_range=0.15,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest')

    train_generator = datagen.flow_from_dataframe(dataframe=tdf, x_col="filename", y_col="label",
                                                  class_mode="categorical", target_size=target_size,
                                                  shuffle=True,
                                                  batch_size=batch_size)

    valid_generator = vdatagen.flow_from_dataframe(dataframe=vdf, x_col="filename", y_col="label",
                                                   class_mode="categorical", target_size=target_size,
                                                   shuffle=True,
                                                   batch_size=batch_size)
    print(tdf)
    print(vdf)
    print(train_generator)
    print(valid_generator)
    STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
    STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

    model.fit_generator(generator=train_generator,
                        validation_data=valid_generator,
                        epochs=100,
                        verbose=1,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_steps=STEP_SIZE_VALID,
                        callbacks=callbacks_list)
    # ......................................................................


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Inpt_conv (Conv2D)              (None, 128, 128, 16) 1216        input_1[0][0]                    
__________________________________________________________________________________________________
Inpt_bn (BatchNormalization)    (None, 128, 128, 16) 64          Inpt_conv[0][0]                  
__________________________________________________________________________________________________
Inpt_relu (Activation)          (None, 128, 128, 16) 0           Inpt_bn[0][0]                    
______________________________________________________________________________________________

Learning rate: 0.001
Epoch 1/100
5/5 [==============================] - 37s 7s/step - loss: 0.9575 - acc: 0.7089 - val_loss: 1235.8418 - val_acc: 0.4023
Learning rate: 0.001
Epoch 2/100
5/5 [==============================] - 25s 5s/step - loss: 0.3109 - acc: 0.9801 - val_loss: 1622.0615 - val_acc: 0.4023
Learning rate: 0.001
Epoch 3/100
5/5 [==============================] - 25s 5s/step - loss: 0.2733 - acc: 0.9906 - val_loss: 1864.0222 - val_acc: 0.4023
Learning rate: 0.001
Epoch 4/100
5/5 [==============================] - 23s 5s/step - loss: 0.2503 - acc: 0.9982 - val_loss: 1515.5694 - val_acc: 0.4023
Learning rate: 0.001
Epoch 5/100
5/5 [==============================] - 21s 4s/step - loss: 0.2530 - acc: 0.9979 - val_loss: 1050.4419 - val_acc: 0.4023
Learning rate: 0.001
Epoch 6/100
5/5 [==============================] - 22s 4s/step - loss: 0.2546 - acc: 0.9969 - val_loss: 705.8777 - val_acc: 0.4023
Learning rate: 0.001
Epoch 7/100
5/5 [==============================] - 21s 4s/step

In [13]:
model.save('model/full_resnet_model_v2.h5')